In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install -q transformers datasets accelerate

import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig
import pandas as pd

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 109.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 104.0 MB/s eta 0:00:00


In [ ]:
train_path = "/content/drive/MyDrive/CosmopediaStories/cosmopedia_stories_translated_tr_02.csv"
train_data = pd.read_csv(train_path)

val_path = "/content/drive/MyDrive/Bitirme/Datasets/validation_dataset.csv"
val_data = pd.read_csv(val_path)

n_total = len(train_data)
chunk_size = n_total // 50

#train_data = train_data.iloc[0:chunk_size*5]
#train_data = train_data.iloc[chunk_size*5:chunk_size*10]
#train_data = train_data.iloc[chunk_size*10:chunk_size*15]
#train_data = train_data.iloc[chunk_size*15:chunk_size*20]

train_texts = train_data["Text"].tolist()
train_dataset = Dataset.from_dict({"text": train_texts})

val_texts = val_data["Text"].tolist()
val_dataset = Dataset.from_dict({"text": val_texts})

In [ ]:
from transformers import AutoConfig
model_name = "unsloth/Llama-3.2-1B"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

config = AutoConfig.from_pretrained(model_name)
model = AutoModelForCausalLM.from_config(config).to(device)

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/889 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

In [ ]:
# Step 4: Tokenization
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )

tokenized_train = train_dataset.map(tokenize_function, batched=True, remove_columns=["text"])
tokenized_val = val_dataset.map(tokenize_function, batched=True, remove_columns=["text"])

Map:   0%|          | 0/228800 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [ ]:
# Step 4: Create DataLoaders
train_loader = DataLoader(tokenized_train, batch_size=16, shuffle=True, collate_fn=lambda x: {
    'input_ids': torch.stack([torch.tensor(f['input_ids']) for f in x]),
    'attention_mask': torch.stack([torch.tensor(f['attention_mask']) for f in x]),
    'labels': torch.stack([torch.tensor(f['input_ids']) for f in x])
})

val_loader = DataLoader(tokenized_val, batch_size=16, shuffle=True, collate_fn=lambda x: {
    'input_ids': torch.stack([torch.tensor(f['input_ids']) for f in x]),
    'attention_mask': torch.stack([torch.tensor(f['attention_mask']) for f in x]),
    'labels': torch.stack([torch.tensor(f['input_ids']) for f in x])
})

In [ ]:
# Step 5: Setup Optimizer
optimizer = AdamW(model.parameters(), lr=2e-4)
print(f"Total transformer layers: {len(model.model.layers)}")
print(next(model.parameters()).dtype)
print(model.dtype)

Total transformer layers: 16
torch.bfloat16
torch.bfloat16


In [ ]:
for i, layer in enumerate(model.model.layers):
    for param in layer.parameters():
        param.requires_grad = False

In [ ]:
layer_group = 3

for i, layer in enumerate(model.model.layers):
    if layer_group == 1 and i < 4:
        for param in layer.parameters():
            param.requires_grad = True
    elif layer_group == 2 and i >= 4 and i < 8:
        for param in layer.parameters():
            param.requires_grad = True
    elif layer_group == 3 and i >= 8 and i < 12:
        for param in layer.parameters():
            param.requires_grad = True
    elif layer_group == 4 and i >= 12 and i < 16:
        for param in layer.parameters():
            param.requires_grad = True

for i, layer in enumerate(model.model.layers):
    trainable = any(p.requires_grad for p in layer.parameters())
    status = "✅" if trainable else "❌"
    print(f"Layer {i}: {status}")

Layer 0: ✅
Layer 1: ✅
Layer 2: ✅
Layer 3: ✅
Layer 4: ✅
Layer 5: ✅
Layer 6: ✅
Layer 7: ✅
Layer 8: ✅
Layer 9: ✅
Layer 10: ✅
Layer 11: ✅
Layer 12: ❌
Layer 13: ❌
Layer 14: ❌
Layer 15: ❌


In [ ]:
for i, layer in enumerate(model.model.layers):
    if i == 5:
        for param in layer.parameters():
            param.requires_grad = True

for i, layer in enumerate(model.model.layers):
    trainable = any(p.requires_grad for p in layer.parameters())
    status = "✅" if trainable else "❌"
    print(f"Layer {i}: {status}")

Layer 0: ✅
Layer 1: ✅
Layer 2: ✅
Layer 3: ✅
Layer 4: ✅
Layer 5: ✅
Layer 6: ✅
Layer 7: ✅
Layer 8: ✅
Layer 9: ✅
Layer 10: ✅
Layer 11: ✅
Layer 12: ✅
Layer 13: ✅
Layer 14: ✅
Layer 15: ✅


In [ ]:
from torch.amp import autocast
from tqdm import tqdm
import torch # Import torch if not already imported in this cell

# To Save Steps
step_flag = 1

# Remove GradScaler
# scaler = GradScaler()

num_epochs = 1
j = 0

total_steps = len(train_loader) * num_epochs
save_steps = [int(total_steps * i / 8) for i in range(1, 8)]
current_step = 0  # Toplam adım sayac

# Training loop with mixed precision
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")

    for batch in progress_bar:
        optimizer.zero_grad()
        batch = {k: v.to(device) for k, v in batch.items()}

        # Use autocast for mixed precision with bfloat16
        with autocast(device_type="cuda", dtype=torch.bfloat16):
            outputs = model(**batch)
            loss = outputs.loss

        # Perform backward pass directly without scaler
        loss.backward()

        # Clip gradients directly without scaler.unscale_
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        # Step optimizer directly without scaler.step
        optimizer.step()

        # No scaler.update() needed when not using GradScaler

        total_loss += loss.item()
        progress_bar.set_postfix({"Loss": loss.item()})

        current_step += 1

        # 🔒 Ara kaydetme noktası
        if current_step in save_steps and step_flag == 1:
            j += 1
            checkpoint_dir = f"/content/drive/MyDrive/Bitirme/Models/Config/Checkpoints/llama-1b-config-cosmopedia-0-1gb.step_{j}"
            model.save_pretrained(checkpoint_dir)
            tokenizer.save_pretrained(checkpoint_dir)
            print(f"🔒 Checkpoint saved at step {current_step}")

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1} Train Loss: {avg_loss:.4f}")

    # Validation phase
    model.eval()  # Set the model to evaluation mode
    total_val_loss = 0
    val_progress_bar = tqdm(val_loader, desc=f"Validation Epoch {epoch+1}/{num_epochs}")

    with torch.no_grad():  # No need to compute gradients for validation
        for batch in val_progress_bar:
            batch = {k: v.to(device) for k, v in batch.items()}

            # Use autocast for mixed precision during validation with bfloat16
            with autocast(device_type="cuda", dtype=torch.bfloat16):
                outputs = model(**batch)
                val_loss = outputs.loss

            total_val_loss += val_loss.item()
            val_progress_bar.set_postfix({"Val Loss": val_loss.item()})

    avg_val_loss = total_val_loss / len(val_loader)
    print(f"Epoch {epoch+1} Validation Loss: {avg_val_loss:.4f}")

final_path = "/content/drive/MyDrive/Bitirme/Models/Config/llama-1b-config-cosmopedia-0-1gb"
model.save_pretrained(final_path)
tokenizer.save_pretrained(final_path)
#torch.save(optimizer.state_dict(), final_path)
#torch.save(model.state_dict(), final_path)

Epoch 1/1:  12%|█▏        | 1787/14300 [14:09<13:39:58,  3.93s/it, Loss=2.15]

🔒 Checkpoint saved at step 1787


Epoch 1/1:  25%|██▌       | 3575/14300 [28:16<9:04:50,  3.05s/it, Loss=1.83]

🔒 Checkpoint saved at step 3575


Epoch 1/1:  37%|███▋      | 5362/14300 [42:20<6:38:23,  2.67s/it, Loss=1.75]

...

Epoch 1/1: 100%|██████████| 14300/14300 [1:52:35<00:00,  2.12it/s, Loss=1.45]


Epoch 1 Train Loss: 1.7528


Validation Epoch 1/1: 100%|██████████| 1/1 [00:00<00:00, 37.56it/s, Val Loss=3.86]


Epoch 1 Validation Loss: 3.8571


('/content/drive/MyDrive/Bitirme/Models/Config/llama-1b-config-cosmopedia-0-1gb/tokenizer_config.json',
 '/content/drive/MyDrive/Bitirme/Models/Config/llama-1b-config-cosmopedia-0-1gb/special_tokens_map.json',
 '/content/drive/MyDrive/Bitirme/Models/Config/llama-1b-config-cosmopedia-0-1gb/tokenizer.json')